In [2]:
import logging
import os
from abc import ABC, abstractmethod
from dataclasses import asdict, dataclass
from datetime import datetime, timedelta
from typing import Callable, List, Tuple

import praw
import tweepy
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
logging.info('No transformation applied.')

In [ ]:
def list_tables(db_name: str):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return [table[0] for table in tables]

# Gọi hàm để in ra danh sách các bảng
tables = list_tables('./data/socialetl.db')
print(tables)

In [ ]:
def read_table(db_name: str, table_name: str):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()
        return rows

# Gọi hàm để đọc dữ liệu từ bảng cụ thể
table_name = 'social_posts'  # Thay 'your_table_name' bằng tên bảng bạn muốn đọc
data_from_table = read_table('./data/socialetl.db', table_name)
print(data_from_table)

In [3]:
REDDIT_CLIENT_ID='9df3vSq_jjssCjJ7nbT_cg'
REDDIT_CLIENT_SECRET='yQxsfT7ClvyHDXebhuvk0w-WOX78eA'
REDDIT_USER_AGENT='ggeddit'

def extract():
    client = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT,
    )
    subreddit = client.subreddit('dataengineering')
    top_subreddit = subreddit.hot(limit=100)
    data = []
    for submission in top_subreddit:
        data.append(
            {
                'title': submission.title,
                'score': submission.score,
                'id': submission.id,
                'url': submission.url,
                'comments': submission.num_comments,
                'created': submission.created,
                'text': submission.selftext,
            }
        )
    return data

In [ ]:
data = extract()

In [4]:
import sqlite3
from contextlib import contextmanager
from typing import Iterator

In [5]:
class DatabaseConnection:
    def __init__(
        self, db_type: str = 'sqlite3', db_file: str = 'data/socialetl.db'
    ) -> None:
        """Class to connect to a database.

        Args:
            db_type (str, optional): Database type.
                Defaults to 'sqlite3'.
            db_file (str, optional): Database file.
                Defaults to 'data/socialetl.db'.
        """
        self._db_type = db_type
        self._db_file = db_file

    @contextmanager
    def managed_cursor(self) -> Iterator[sqlite3.Cursor]:
        """Function to create a managed database cursor.

        Yields:
            sqlite3.Cursor: A sqlite3 cursor.
        """
        if self._db_type == 'sqlite3':
            _conn = sqlite3.connect(self._db_file)
            cur = _conn.cursor()
            try:
                yield cur
            finally:
                _conn.commit()
                cur.close()
                _conn.close()


In [8]:
def db_factory(
    db_type: str = 'sqlite3', db_file: str = 'data/socialetl.db'
) -> DatabaseConnection:
    """Function to create an ETL object.

    Args:
        db_type (str, optional): Database type.
            Defaults to 'sqlite3'.
        db_file (str, optional): Database file.
            Defaults to 'data/socialetl.db'.

    Returns:
        DatabaseConnection: A DatabaseConnection object.
    """
    return DatabaseConnection(db_type=db_type, db_file=db_file)